In [1]:
from openai import OpenAI
from pydantic import BaseModel
import os
from dotenv import load_dotenv
# Import thư viện, đối với GPT (ChatGPT) hay Claude (mô hình chuyên dùng trong CLINE),... cũng có các thư viện tương tự.
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
from tools.web_search import web_search 
from tools.product_tool import product_search

d:\MISA\AI\src\HiveSpace.ChatboxAI\apis\agents\tools\web_search.py:18: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(input, max_results=5, region="vn-vi")


[{'title': 'MISA nhận nhiệm vụ chiến lược quốc gia làm chủ công nghệ AI', 'href': 'https://www.misa.vn/150413/misa-nhan-nhiem-vu-chien-luoc-quoc-gia-lam-chu-cong-nghe-ai/', 'body': 'Jan 15, 2025 · Đây không chỉ là một nhiệm vụ chiến lược tiên phong mà còn là sứ mệnh lịch sử mang dấu ấn trí tuệ Việt, kiến tạo công nghệ AI …'}, {'title': 'MISA nhận nhiệm vụ làm chủ công nghệ AI - VnExpress', 'href': 'https://vnexpress.net/misa-nhan-nhiem-vu-lam-chu-cong-nghe-ai-4840523.html', 'body': 'Jan 19, 2025 · "Trong 5 năm tới, chúng tôi sẽ đầu tư 2.500 tỷ đồng để xây dựng mô hình ngôn ngữ lớn LLM dành cho AI "Make in Vietnam". Mô …'}, {'title': "Ứng dụng AI trong chuyển đổi số: MISA và sứ mệnh quốc gia 'Make in …", 'href': 'https://nguoiquansat.vn/ung-dung-ai-trong-chuyen-doi-so-misa-va-su-menh-quoc-gia-make-in-vietnam-225919.html', 'body': 'MISA vừa được Tổng Bí thư Tô Lâm giao trọng trách quốc gia làm chủ công nghệ AI, với cam kết đầu tư 2.500 tỷ đồng trong 5 năm tới để xây …'}, {'title': '#misa

In [3]:
# Load biến từ .env vào môi trường
load_dotenv()

# Lấy API key
api_key = os.getenv("api_key")

In [4]:
# Khởi tạo một Basic Agent
basic_agent = ChatGoogleGenerativeAI(
    model= "gemini-2.0-flash",    # Model ngôn ngữ lớn"
    temperature=1.0,              # Mức độ sáng tạo của model, từ 0 tới 1.
    max_tokens=None,              # Giới hạn token của Input, Output. Thường nên để tối đa 32K.
    timeout=None,
    max_retries=2,
    google_api_key=api_key        #API key đã lấy ở trên
)

In [5]:
# Create LLM class
llm = ChatGoogleGenerativeAI(
    model= "gemini-2.0-flash", # replace with "gemini-2.0-flash"
    temperature=1.0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    google_api_key=api_key
)

# Bind tools to the model
react_agent = llm.bind_tools([web_search, product_search])

In [6]:
"""
Các bước ghép nối luồng hoạt động của AI Agent
"""

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [7]:
import json
from langchain_core.messages import ToolMessage, SystemMessage
from langchain_core.runnables import RunnableConfig

tools = [web_search, product_search]

tools_by_name = {tool.name: tool for tool in tools}


# Define our tool node
def call_tool(state: State):
    outputs = []
    # Iterate over the tool calls in the last message
    for tool_call in state["messages"][-1].tool_calls:
        # Get the tool by name
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=tool_result,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}

def call_model(
    state: State,
    config: RunnableConfig,
):
    # Invoke the model with the system prompt and the messages
    response = react_agent.invoke(state["messages"], config)
    # We return a list, because this will get added to the existing messages state using the add_messages reducer
    return {"messages": [response]}


# Define the conditional edge that determines whether to continue or not
def should_continue(state: State):
    messages = state["messages"]
    # If the last message is not a tool call, then we finish
    if not messages[-1].tool_calls:
        return "end"
    # default to continue
    return "continue"

In [8]:
from langgraph.graph import StateGraph, END

# Define a new graph with our state
workflow = StateGraph(State)

# 1. Add our nodes
workflow.add_node("llm", call_model)
workflow.add_node("tools",  call_tool)
# 2. Set the entrypoint as `agent`, this is the first node called
workflow.set_entry_point("llm")
# 3. Add a conditional edge after the `llm` node is called.
workflow.add_conditional_edges(
    # Edge is used after the `llm` node is called.
    "llm",
    # The function that will determine which node is called next.
    should_continue,
    # Mapping for where to go next, keys are strings from the function return, and the values are other nodes.
    # END is a special node marking that the graph is finish.
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # Otherwise we finish.
        "end": END,
    },
)
# 4. Add a normal edge after `tools` is called, `llm` node is called next.
workflow.add_edge("tools", "llm")

# Now we can compile and visualize our graph
react_agent_graph = workflow.compile()

In [9]:
# Viết lại hàm gọi react_agent_graph
def ask_react_agent(messages: list):
    print("Human: " + messages[-1]["content"])

    for event in react_agent_graph.stream({"messages": messages}):
        for key, value in event.items():
            if key != "llm" or value["messages"][-1].content == "":
                continue
            print("AI:", value["messages"][-1].content)

In [10]:
# Thử nghiệm
ask_react_agent([{
                "role": "system",
                "content": "Bạn là AVA, trợ lý số của công ty cổ phần MISA.",
            },
             {
                "role": "system",
                "content": f'''Thông tin bổ sung:
                            - Thời gian hiện tại: 08-2025''',
            },
            {
                "role": "user",
                "content": "Lấy cho tôi danh sách sản phẩm",
            }])

Human: Lấy cho tôi danh sách sản phẩm
AI: Dưới đây là danh sách sản phẩm hiện có:

*   **Laptop Dell XPS 13:** Giá 1500, thuộc danh mục Điện tử, còn hàng.
*   **iPhone 15 Pro:** Giá 1200, thuộc danh mục Điện tử, hết hàng.
*   **Bàn phím cơ Keychron K2:** Giá 90, thuộc danh mục Phụ kiện, còn hàng.
*   **Ghế công thái học Sihoo:** Giá 250, thuộc danh mục Nội thất, còn hàng.
*   **Tai nghe Sony WH-1000XM5:** Giá 400, thuộc danh mục Âm thanh, hết hàng.
